## U-Net

In [1]:
import torch
from torch import nn

In [ ]:
## ConvBlock
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.convs = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),  ## padding=1 keep input and output have same height and width
            nn.BatchNorm2d(out_ch),
            nn.ReLU(),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU()
        )
    
    def forward(self, x):
        return self.convs(x)

In [ ]:
## U-Net
class UNet(nn.Module):
    def __init__(self, in_ch=1):
        super().__init__()
        self.down1=ConvBlock(in_ch, 64)
        self.down2=ConvBlock(64, 128)
        self.bot1=ConvBlock(128, 256)  ## deeper squeeze
        self.up2=ConvBlock(128+256, 128)  ## concatenate with down2 in dim1
        self.up1=ConvBlock(128+64, 64)  ## concatenate with down1 in dim1
        self.out=nn.Conv2d(64, in_ch, 1)

        self.maxpool=nn.MaxPool2d(2)  
        self.upsample=nn.Upsample(scale_factor=2, mode="bilinear")  ## twice as before
    
    def forward(self, x):
        x1=self.down1(x)
        x=self.maxpool(x1)
        x2=self.down2(x)
        x=self.maxpool(x2)
        x=self.bot1(x)

        x=self.upsample(x)
        x=torch.cat([x, x2], dim=1)
        x=self.up2(x)
        x=self.upsample(x)
        x=torch.cat([x, x1], dim=1)
        x=self.up1(x)
        x=self.out(x)

        return x

In [6]:
## using random data for demo
model=UNet()
x=torch.randn(10, 1, 28, 28)
y=model(x)
print(y.shape)

torch.Size([10, 1, 28, 28])
